In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image
from utils import bn_update, moving_average, copy_model


In [2]:
!ls /home/chec/data/bengali

ls: cannot access '/home/chec/data/bengali': No such file or directory


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/mnt/chicm/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
from torch.utils.data.sampler import WeightedRandomSampler

def class_balanced_sampler(labels):
    class_counts = np.bincount(labels)
    total_samples = len(labels)
    sample_weights = np.zeros_like(labels).astype(np.float32)
    for idx, label in enumerate(labels):
        sample_weights[idx] = total_samples / class_counts[label]
    # return sample_weights
    sampler = WeightedRandomSampler(weights=sample_weights,
        num_samples=total_samples)
    return sampler


In [8]:
HEIGHT = 137
WIDTH = 236

In [9]:
#import albumentations as albu
def get_train_augs():
    return RandAugment(n=2, m=27)

In [10]:
#plt.imshow(x)
finetune_class = 'vowel_diacritic'

In [11]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        orig_img = img.copy()
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            #img = augs(image=img)['image']
            img = np.asarray(augs(Image.fromarray(img)))
        
        img = np.expand_dims(img, axis=-1)
        orig_img = np.expand_dims(orig_img, axis=-1)
        
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        orig_img = transforms.functional.to_tensor(orig_img)
        
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        elif self.train_mode:
            return img, orig_img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
                    
    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    print('fold:', ifold)
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    sampler = class_balanced_sampler(train[finetune_class])
    print('sampler length: {}'.format(sampler.weights.shape))

    
    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=False, num_workers=8, drop_last=False, pin_memory=True, sampler=sampler)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(
        val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False, pin_memory=True)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [12]:
#train_loader, val_loader = get_train_val_loaders()

In [13]:
#for x in train_loader:
#    print(x)
#    break

# model

In [14]:
#import pretrainedmodels

In [15]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [16]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]


In [17]:
class BengaliNetOld(nn.Module):
    def __init__(self, backbone_name='se_resnext50_32x4d'):
        super(BengaliNetOld, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.n_word = 1295
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant + self.n_word
        
        self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
        self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)
        
        self.num_p2_features = self.backbone.layer2[-1].se_module.fc2.out_channels
        self.num_p3_features = self.backbone.layer3[-1].se_module.fc2.out_channels
        self.p2_head = nn.Conv2d(self.num_p2_features, self.num_p2_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.p3_head = nn.Conv2d(self.num_p3_features, self.num_p3_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(self.num_p2_features * 4)
        self.bn3 = nn.BatchNorm2d(self.num_p3_features * 4)
        self.act2 = Swish()
        self.act3 = Swish()
        
        self.fc_aux1 = nn.Linear(self.num_p3_features * 4, self.num_classes)
        self.fc_aux2 = nn.Linear(self.num_p2_features * 4, self.num_classes)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        for fc in [self.fc, self.fc_aux1, self.fc_aux2]:
            nn.init.zeros_(fc.bias.data)

        print('init model4')
        
    def features(self, x):
        x = self.backbone.layer0(x); #print(x.size())
        x = self.backbone.layer1(x); #print(x.size())
        x = self.backbone.layer2(x); p2 = x; p2 = self.p2_head(p2); p2 = self.bn2(p2); p2 = self.act2(p2) #print(x.size())
        x = self.backbone.layer3(x); p3 = x; p3 = self.p3_head(p3); p3 = self.bn3(p3); p3 = self.act3(p3) #print(x.size())
        x = self.backbone.layer4(x); #print(x.size())
        return x, p2, p3
        
    def logits(self, x, p2, p3):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = torch.flatten(x, 1)
        
        p2 = self.avg_pool(p2)
        p2 = torch.flatten(p2, 1)
        
        p3 = self.avg_pool(p3)
        p3 = torch.flatten(p3, 1)
        return self.fc(x), self.fc_aux1(p3), self.fc_aux2(p2)
    
    def forward(self, x):
        x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        x, p2, p3 = self.features(x)
        x, logits_aux1, logits_aux2 = self.logits(x, p2, p3)

        return x, logits_aux1, logits_aux2

In [18]:
class BengaliNet(nn.Module):
    def __init__(self, backbone_name='se_resnext50_32x4d'):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.n_word = 1295
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant + self.n_word
        
        self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
        
        self.num_p2_features = self.backbone.layer2[-1].se_module.fc2.out_channels
        self.num_p3_features = self.backbone.layer3[-1].se_module.fc2.out_channels
        self.p2_head = nn.Conv2d(self.num_p2_features, self.num_p2_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.p3_head = nn.Conv2d(self.num_p3_features, self.num_p3_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(self.num_p2_features * 4)
        self.bn3 = nn.BatchNorm2d(self.num_p3_features * 4)
        self.act2 = Swish()
        self.act3 = Swish()
        
        
        #self.fc = nn.Linear(self.num_features, num_total_classes)
        #self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)
        
        self.fc_g = nn.Linear(self.backbone.last_linear.in_features, 168)
        self.fc_v = nn.Linear(self.backbone.last_linear.in_features, 11)
        self.fc_c = nn.Linear(self.backbone.last_linear.in_features, 7)
        self.fc_w = nn.Linear(self.backbone.last_linear.in_features, 1295)

        
        self.fc_aux1 = nn.Linear(self.num_p3_features * 4, self.num_classes)
        self.fc_aux2 = nn.Linear(self.num_p2_features * 4, self.num_classes)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        #for fc in [self.fc, self.fc_aux1, self.fc_aux2]:
        #    nn.init.zeros_(fc.bias.data)
        for fc in [self.fc_g, self.fc_v, self.fc_c, self.fc_w, self.fc_aux1, self.fc_aux2]:
            nn.init.zeros_(fc.bias.data)

        print('init model4')
        
    def features(self, x):
        x = self.backbone.layer0(x); #print(x.size())
        x = self.backbone.layer1(x); #print(x.size())
        x = self.backbone.layer2(x); p2 = x; p2 = self.p2_head(p2); p2 = self.bn2(p2); p2 = self.act2(p2) #print(x.size())
        x = self.backbone.layer3(x); p3 = x; p3 = self.p3_head(p3); p3 = self.bn3(p3); p3 = self.act3(p3) #print(x.size())
        x = self.backbone.layer4(x); #print(x.size())
        return x, p2, p3
        
    def logits(self, x, p2, p3):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = torch.flatten(x, 1)
        
        p2 = self.avg_pool(p2)
        p2 = torch.flatten(p2, 1)
        
        p3 = self.avg_pool(p3)
        p3 = torch.flatten(p3, 1)
        
        #logits = self.fc(x)
        g, v, c, w = self.fc_g(x), self.fc_v(x), self.fc_c(x), self.fc_w(x)
        logits = torch.cat([g, v, c, w], 1)
        
        return logits, self.fc_aux1(p3), self.fc_aux2(p2)
    
    def forward(self, x):
        x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        x, p2, p3 = self.features(x)
        x, logits_aux1, logits_aux2 = self.logits(x, p2, p3)

        return x, logits_aux1, logits_aux2
    
    def set_mode(self, mode, is_freeze_bn=True):
        self.mode = mode
        if mode in ['eval', 'valid', 'test']:
            self.eval()

        elif mode in ['train']:
            self.train()
        if is_freeze_bn == True:  ##freeze
            for m in self.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()
                    m.weight.requires_grad = False
                    m.bias.requires_grad = False


In [19]:
!ls ./model4-ckps/se_resnext50_32x4d

model4_se_resnext50_fold0_224_cv9976.pth
model4_se_resnext50_fold0_224_cv9977.pth
model4_se_resnext50_fold0_224_cv9978.pth
model4_se_resnext50_fold0_224_cv998106.pth
model4_se_resnext50_fold0_224_swa_cv998273.pth
model4_se_resnext50_fold4_224_cv997979.pth
model4_se_resnext50_fold4_224_swa_cv998285.pth


In [20]:
#!mkdir ./model4-finetune-ckps/se_resnext50_32x4d

In [21]:
def convert_model():
    model_old = BengaliNetOld()
    model_old.load_state_dict(torch.load('./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224_cv998106.pth'))    
    
    model = BengaliNet()
    model.load_state_dict(torch.load('./model4-ckps/se_resnext50_32x4d/model4_se_resnext50_fold0_224_cv998106.pth'), strict=False)
    model.fc_g.weight.data = model_old.fc.weight.data[:168] 
    model.fc_v.weight.data = model_old.fc.weight.data[168:168+11] 
    model.fc_c.weight.data = model_old.fc.weight.data[168+11:168+11+7] 
    model.fc_w.weight.data = model_old.fc.weight.data[168+11+7:] 
    model = model.cuda()
    model = amp.initialize(model, opt_level='O1')
    model = nn.DataParallel(model)
    validate(model, val_loader)
    torch.save(model.module.state_dict(), './model4-finetune-ckps/se_resnext50_32x4d/model4_finetune_res50_fold0.pth')

In [22]:
#convert_model()

In [23]:
MODEL_DIR = './model4-finetune-ckps'
def create_model(args):
    model = BengaliNet()
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [24]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [25]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [26]:
round(1/9, 6)

0.111111

In [27]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(y[:, 0], preds0, average='macro')
    recall_vowel = sklearn.metrics.recall_score(y[:, 1], preds1, average='macro')
    recall_consonant = sklearn.metrics.recall_score(y[:, 2], preds2, average='macro')
    recall_word = sklearn.metrics.recall_score(y[:, 3], preds3, average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    old_recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    old_recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    old_recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    old_recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    old_scores = [old_recall_grapheme, old_recall_vowel, old_recall_consonant]
    old_final_recall_score = np.average(old_scores, weights=[2, 1, 1])

    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)  
    
    metrics['old_recall'] = round(old_final_recall_score, 6)
    metrics['old_recall_grapheme'] = round(old_recall_grapheme, 6)
    
    return metrics

In [28]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()

In [29]:
def validate(model, val_loader):
    #model.eval()
    model.module.set_mode('valid', is_freeze_bn=True)

    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, _, _ = model(x)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [30]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [31]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [32]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [33]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [34]:
def rand_bbox_new(size, lam):
    H = size[2]
    W = size[3]

    x_margin_rate = 0.2

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * (1-x_margin_rate*2) * cut_rat)
    cut_h = np.int(H * cut_rat)
    
    min_x_center = np.int(W * x_margin_rate + cut_w / 2)
    max_x_center = np.int(W * (1-x_margin_rate) - cut_w / 2)
    #print(min_x_center, max_x_center, lam, cut_w)
    min_y_center = cut_h // 2
    max_y_center = H - cut_h // 2
    if max_y_center == min_y_center:
        max_y_center += 1

    # uniform
    cx = np.random.randint(min_x_center, max_x_center)
    cy = np.random.randint(min_y_center, max_y_center)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    
    #print(bbx1, bbx2, bby1, bby2)

    return bbx1, bby1, bbx2, bby2

In [35]:
np.random.random()

0.33354110245466095

In [36]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from gridmask import GridMask

In [37]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [38]:
def freeze_for_finetune(net):
    for param in net.parameters():
        param.requires_grad = False
    if finetune_class == 'grapheme_root':
        for param in net.fc_g.parameters():
            param.requires_grad = True
    elif finetune_class == 'vowel_diacritic':
        for param in net.fc_v.parameters():
            param.requires_grad = True
    else:
        for param in net.fc_c.parameters():
            param.requires_grad = True

In [39]:
import time
def train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid):
    train_loss = 0
    bg = time.time()

    for batch_idx, (img, orig_img, targets) in enumerate(train_loader):
        img, orig_img, targets  = img.cuda(), orig_img.cuda(), targets.cuda()
        batch_size = img.size(0)
        r = np.random.rand()

        #if True:
        #    outputs, outputs_aux1, outputs_aux2 = model(orig_img)
        #    loss = criterion(outputs, targets)
            #loss_aux1 = criterion(outputs_aux1, targets)
            #loss_aux2 = criterion(outputs_aux2, targets)
            #loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        if r < 0.5:
            # generate mixed sample
            lam = np.random.beta(args.beta, args.beta)
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = targets
            target_b = targets[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            #img[:, :, bby1:bby2, bbx1:bbx2] = img[rand_index, :, bby1:bby2, bbx1:bbx2] #for new cutmix
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            # compute output
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            #loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
            #loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
            #loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        elif r > 0.5:
            img = grid(img)
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss = criterion(outputs, targets)
            #loss_aux1 = criterion(outputs_aux1, targets)
            #loss_aux2 = criterion(outputs_aux2, targets)
            #loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        else:
            orig_img, targets = mixup(orig_img, targets)
            outputs, outputs_aux1, outputs_aux2 = model(orig_img)
            loss = mixup_criterion(outputs, targets)
            #loss_aux1 = mixup_criterion(outputs_aux1, targets)
            #loss_aux2 = mixup_criterion(outputs_aux2, targets)
            #loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
            #loss = criterion(outputs, targets)

        optimizer.zero_grad()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        lr_scheduler(optimizer, batch_idx, epoch)
        optimizer.step()            
        
        current_lr = get_lrs(optimizer)

        train_loss += loss.item()
        print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} | {:.2f} |'.format(
            epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
            loss.item(), train_loss/(batch_idx+1), (time.time() - bg) / 60), end='')


In [40]:
import copy
best_metrics = 0.
best_metrics_swa = 0.


def validate_and_save(model, model_file, val_loader, save=False):
    global best_metrics
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    print('current best_metrics:', best_metrics)
    if val_metrics[best_key] > best_metrics:
        print('saving...')
        best_metrics = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.module.set_mode('train', is_freeze_bn=True)

def validate_and_save_swa(model, model_file, val_loader, save=False):
    global best_metrics_swa
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics_swa:
        best_metrics_swa = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.module.set_mode('train', is_freeze_bn=True)



def train(args):
    model, model_file = create_model(args)
    model = model.cuda()
    
    freeze_for_finetune(model)

    swa_args = copy.deepcopy(args)
    swa_args.ckp_name = args.ckp_name + '_swa'
    swa_model, swa_model_file = create_model(swa_args)
    swa_model = swa_model.cuda()

    optimizer = make_optimizer(model)
    lr_scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    [model, swa_model], optimizer = amp.initialize([model, swa_model], optimizer, opt_level="O1",verbosity=0)
    #[model, swa_model], optimizer = amp.initialize(
    #    [model, swa_model], optimizer, opt_level="O2",verbosity=0, keep_batchnorm_fp32=True)
    
    #opt_level="O2", keep_batchnorm_fp32=True
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        swa_model = nn.DataParallel(swa_model)
    
    validate_and_save(model, model_file, val_loader, save=False)

    swa_model_loaded = False
    if os.path.exists(swa_model_file):
        swa_model_loaded = True
        validate_and_save_swa(swa_model, swa_model_file, val_loader, save=False)
        
    model.module.set_mode('train', is_freeze_bn=True)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

        for epoch in range(args.start_epoch, args.num_epochs):
            grid.set_prob(epoch, args.st_epochs)
            train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid)
            validate_and_save(model, model_file, val_loader, save=True)
            
            if (epoch+1) == args.swa_start and cycle == 1:
                if not swa_model_loaded:
                    copy_model(swa_model, model)
                #swa_n = 0
                swa_n = args.swa_n
            if (epoch+1) >= args.swa_start and (epoch+1) % args.swa_freq == 0:
                print('SWA>>>:')
                moving_average(swa_model, model, 1.0 / (swa_n + 1))
                swa_n += 1
                bn_update(train_loader, swa_model)
                validate_and_save_swa(swa_model, swa_model_file, val_loader, save=True)

        #args.base_lr = 2e-4
        #args.num_epochs = 40
        #args.warmup_epochs = 1

        # reset scheduler at each cycle
        lr_scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    del model
    del swa_model

In [41]:
#model, model_file = create_model(cfg)
#model(torch.randn(2,1,137,236))[0].size()

In [42]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'model4_finetune_res50_fold0.pth'
args.base_lr = 8e-5
args.num_epochs = 25
args.start_epoch = 0
args.warmup_epochs = 5
args.num_cycles = 1
args.batch_size = 1536
args.val_batch_size = 1024
args.st_epochs = 5

args.swa_start = 3000
args.swa_freq = 3
args.swa_n = 5

args.beta = 1.0
args.cutmix_prob = 0.5

In [45]:
for i in range(100):
    for x in ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic']:
        finetune_class = x
        #dataset use finetune_class
        train_loader, val_loader = get_train_val_loaders(
            batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=0)
        train(args)

fold: 0
(200840, 6)
(200840, 32332)
(160596, 6) (40244, 6)
sampler length: torch.Size([160596])
init model4
model file: ./model4-finetune-ckps/se_resnext50_32x4d/model4_finetune_res50_fold0.pth, exist: True
loading ./model4-finetune-ckps/se_resnext50_32x4d/model4_finetune_res50_fold0.pth...
init model4
model file: ./model4-finetune-ckps/se_resnext50_32x4d/model4_finetune_res50_fold0.pth_swa, exist: False

val: {'recall': 0.997562, 'recall_grapheme': 0.99736, 'recall_vowel': 0.998099, 'recall_consonant': 0.997429, 'recall_word': 0.997165, 'acc_grapheme': 0.997391, 'acc_vowel': 0.998683, 'acc_consonant': 0.998857, 'acc_word': 0.997217, 'old_recall': 0.998109, 'old_recall_grapheme': 0.997877, 'loss_grapheme': 0.012186, 'loss_vowel': 0.005505, 'loss_consonant': 0.004546, 'loss_word': 0.012468}
current best_metrics: 0.0
saving...
CYCLE: 1
    0 | 0.000016 | 089460/160596 | 0.0590 | 3.6460 | 1.51 ||
val: {'recall': 0.99757, 'recall_grapheme': 0.997377, 'recall_vowel': 0.998099, 'recall_conso

   15 | 0.000023 | 089460/160596 | 7.1811 | 2.6028 | 2.67 ||
val: {'recall': 0.997679, 'recall_grapheme': 0.997595, 'recall_vowel': 0.998099, 'recall_consonant': 0.997429, 'recall_word': 0.997165, 'acc_grapheme': 0.997441, 'acc_vowel': 0.998683, 'acc_consonant': 0.998857, 'acc_word': 0.997217, 'old_recall': 0.997733, 'old_recall_grapheme': 0.997125, 'loss_grapheme': 0.022326, 'loss_vowel': 0.005505, 'loss_consonant': 0.004546, 'loss_word': 0.012468}
current best_metrics: 0.997686
   16 | 0.000019 | 089460/160596 | 0.1036 | 3.4135 | 2.41 ||
val: {'recall': 0.99767, 'recall_grapheme': 0.997575, 'recall_vowel': 0.998099, 'recall_consonant': 0.997429, 'recall_word': 0.997165, 'acc_grapheme': 0.997441, 'acc_vowel': 0.998683, 'acc_consonant': 0.998857, 'acc_word': 0.997217, 'old_recall': 0.997749, 'old_recall_grapheme': 0.997158, 'loss_grapheme': 0.023041, 'loss_vowel': 0.005505, 'loss_consonant': 0.004546, 'loss_word': 0.012468}
current best_metrics: 0.997686
   17 | 0.000015 | 089460/16059

    5 | 0.000069 | 089460/160596 | 4.9848 | 3.6579 | 2.60 |||
val: {'recall': 0.99784, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998099, 'recall_consonant': 0.998043, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998683, 'acc_consonant': 0.998832, 'acc_word': 0.997217, 'old_recall': 0.997414, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.005505, 'loss_consonant': 0.009676, 'loss_word': 0.012468}
current best_metrics: 0.997849
    6 | 0.000066 | 089460/160596 | 0.1138 | 3.4492 | 2.55 ||
val: {'recall': 0.997848, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998099, 'recall_consonant': 0.998077, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998683, 'acc_consonant': 0.998857, 'acc_word': 0.997217, 'old_recall': 0.997722, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.005505, 'loss_consonant': 0.012346, 'loss_word': 0.012468}
current best_metrics: 0.997849
    7 | 0.000061 | 089460/160596

   21 | 0.000003 | 089460/160596 | 7.8203 | 3.3582 | 2.63 |||
val: {'recall': 0.998004, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998099, 'recall_consonant': 0.998702, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998683, 'acc_consonant': 0.998932, 'acc_word': 0.997217, 'old_recall': 0.997428, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.005505, 'loss_consonant': 0.050148, 'loss_word': 0.012468}
current best_metrics: 0.998012
   22 | 0.000001 | 089460/160596 | 0.1398 | 3.1655 | 2.62 ||
val: {'recall': 0.998012, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998099, 'recall_consonant': 0.998732, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998683, 'acc_consonant': 0.998956, 'acc_word': 0.997217, 'old_recall': 0.997429, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.005505, 'loss_consonant': 0.050294, 'loss_word': 0.012468}
current best_metrics: 0.998012
   23 | 0.000000 | 089460/16059

   11 | 0.000043 | 089460/160596 | 0.1954 | 3.7387 | 2.65 ||
val: {'recall': 0.998092, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998421, 'recall_consonant': 0.998732, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998758, 'acc_consonant': 0.998956, 'acc_word': 0.997217, 'old_recall': 0.997368, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.05795, 'loss_consonant': 0.050024, 'loss_word': 0.012468}
current best_metrics: 0.998133
   12 | 0.000038 | 089460/160596 | 0.2479 | 3.4432 | 2.63 ||
val: {'recall': 0.998099, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998449, 'recall_consonant': 0.998732, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998807, 'acc_consonant': 0.998956, 'acc_word': 0.997217, 'old_recall': 0.997374, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.059064, 'loss_consonant': 0.050024, 'loss_word': 0.012468}
current best_metrics: 0.998133
   13 | 0.000033 | 089460/160596 

...

   19 | 0.000008 | 089460/160596 | 4.4728 | 2.9302 | 2.83 ||
val: {'recall': 0.9981, 'recall_grapheme': 0.997544, 'recall_vowel': 0.998582, 'recall_consonant': 0.998732, 'recall_word': 0.997165, 'acc_grapheme': 0.997341, 'acc_vowel': 0.998857, 'acc_consonant': 0.998956, 'acc_word': 0.997217, 'old_recall': 0.99721, 'old_recall_grapheme': 0.996841, 'loss_grapheme': 0.046759, 'loss_vowel': 0.048159, 'loss_consonant': 0.050024, 'loss_word': 0.012468}
current best_metrics: 0.998133
   20 | 0.000005 | 089460/160596 | 0.2988 | 3.3422 | 2.70 ||
val: {'recall': 0.9981, 'recall_grapheme': 0.997544, 'recall_vowel': 0.998582, 'recall_consonant': 0.998732, 'recall_word': 0.997165, 'acc_grapheme': 0.997341, 'acc_vowel': 0.998857, 'acc_consonant': 0.998956, 'acc_word': 0.997217, 'old_recall': 0.997229, 'old_recall_grapheme': 0.996879, 'loss_grapheme': 0.046966, 'loss_vowel': 0.048159, 'loss_consonant': 0.050024, 'loss_word': 0.012468}
current best_metrics: 0.998133
   21 | 0.000003 | 089460/160596 | 

    9 | 0.000052 | 089460/160596 | 0.2206 | 3.8988 | 2.64 ||
val: {'recall': 0.99813, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998582, 'recall_consonant': 0.998721, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998857, 'acc_consonant': 0.998907, 'acc_word': 0.997217, 'old_recall': 0.997371, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.048159, 'loss_consonant': 0.060667, 'loss_word': 0.012468}
current best_metrics: 0.998138
   10 | 0.000048 | 089460/160596 | 5.5379 | 3.6922 | 2.63 ||
val: {'recall': 0.99813, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998582, 'recall_consonant': 0.998721, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998857, 'acc_consonant': 0.998907, 'acc_word': 0.997217, 'old_recall': 0.997371, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.048159, 'loss_consonant': 0.060569, 'loss_word': 0.012468}
current best_metrics: 0.998138
   11 | 0.000043 | 089460/160596 |

init model4
model file: ./model4-finetune-ckps/se_resnext50_32x4d/model4_finetune_res50_fold0.pth_swa, exist: False

val: {'recall': 0.998145, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998582, 'recall_consonant': 0.998781, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998857, 'acc_consonant': 0.998932, 'acc_word': 0.997217, 'old_recall': 0.997673, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.048159, 'loss_consonant': 0.061712, 'loss_word': 0.012468}
current best_metrics: 0.998145
CYCLE: 1
    0 | 0.000016 | 089460/160596 | 0.1771 | 3.5216 | 1.68 ||
val: {'recall': 0.998108, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998433, 'recall_consonant': 0.998781, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998782, 'acc_consonant': 0.998932, 'acc_word': 0.997217, 'old_recall': 0.997661, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.048681, 'loss_consonant': 0.061712, 'loss_word': 0.012468

   16 | 0.000019 | 089460/160596 | 0.2698 | 3.5444 | 2.94 ||
val: {'recall': 0.998137, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998549, 'recall_consonant': 0.998781, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998782, 'acc_consonant': 0.998932, 'acc_word': 0.997217, 'old_recall': 0.997633, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.061039, 'loss_consonant': 0.061712, 'loss_word': 0.012468}
current best_metrics: 0.998145
   17 | 0.000015 | 089460/160596 | 0.2856 | 3.2675 | 2.82 ||
val: {'recall': 0.998144, 'recall_grapheme': 0.997608, 'recall_vowel': 0.998577, 'recall_consonant': 0.998781, 'recall_word': 0.997165, 'acc_grapheme': 0.99749, 'acc_vowel': 0.998832, 'acc_consonant': 0.998932, 'acc_word': 0.997217, 'old_recall': 0.997662, 'old_recall_grapheme': 0.997197, 'loss_grapheme': 0.018819, 'loss_vowel': 0.06047, 'loss_consonant': 0.061712, 'loss_word': 0.012468}
current best_metrics: 0.998145
   18 | 0.000011 | 089460/160596 

KeyboardInterrupt: 

In [42]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=0)

fold: 0
(200840, 6)
(200840, 32332)
(160596, 6) (40244, 6)
sampler length: torch.Size([160596])
